In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [ ]:
binaaz_data = pd.read_csv("binaAzData.csv")

In [ ]:
binaaz_data.head()

,Price,Sale_Type,Location,Number_of_rooms,KvM,Floor
0,130000.0,Satış,Əhmədli m.,2.0,65 m²,6.0
1,115000.0,Satış,Sabunçu r.,3.0,110 m²,8.0
2,92000.0,Satış,20 Yanvar m.,2.0,51 m²,20.0
3,435000.0,Satış,Mərdəkan q.,0.0,200 m²,0.0
4,250000.0,Aylıq,Şah İsmayıl Xətai m.,3.0,80.9 m²,16.0


In [ ]:
df=binaaz_data.copy()

In [ ]:
df['Location'] = df['Location'].apply(lambda x: x.replace('r.', '') if 'r.' in x else x)
df['Location'] = df['Location'].apply(lambda x: x.replace('m.', '') if 'm.' in x else x)
df['Location'] = df['Location'].apply(lambda x: x.replace('q.', '') if 'q.' in x else x)
df['Location'] = df['Location'].str.strip()

In [ ]:
df.head()

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
df['KvM_']=df['KvM'].str.split(' ',expand=True)[0]

In [ ]:
df=df.drop('KvM',axis=1)

In [ ]:
df['KvM_']=df['KvM_'].astype(float)

In [ ]:
df['Price']=df['Price'].astype(int)


In [ ]:
price=df['Price']


In [ ]:
df1=df[df['Sale_Type']=='Satış']


In [ ]:
Q1=price.quantile(0.25) 
Q3=price.quantile(0.75)
IQR=Q3-Q1 

In [ ]:
lower_limit=Q1-1.5*IQR
upper_limit=Q3+1.5*IQR

In [ ]:
df1=df1[(df1['Price']<=upper_limit)&(df1['Price']>=Q1)]


In [ ]:
kvm_=df['KvM_']


In [ ]:
Q1=kvm_.quantile(0.25) 
Q3=kvm_.quantile(0.75)
IQR=Q3-Q1 

In [ ]:
lower_limit_kvm=Q1-1.5*IQR
upper_limit_kvm=Q3+1.5*IQR

In [ ]:
df1=df1[df1['KvM_']<=upper_limit_kvm]


In [ ]:

number_rooms=df1['Number_of_rooms']


In [ ]:
Q1=number_rooms.quantile(0.25) 
Q3=number_rooms.quantile(0.75)
IQR=Q3-Q1

In [ ]:
lower_limit_rooms=Q1-1.5*IQR
upper_limit_rooms=Q3+1.5*IQR

In [ ]:
upper_limit_rooms=round(upper_limit_rooms)


In [ ]:
df1=df1[df1['Number_of_rooms']<=upper_limit_rooms]


In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
location_encoder=LabelEncoder()


In [ ]:
df1['Location']=location_encoder.fit_transform(df1['Location'])

In [ ]:
df1.head()

,Price,Sale_Type,Location,Number_of_rooms,Floor,KvM_
0,130000,Satış,141,2.0,6.0,65.0
1,115000,Satış,94,3.0,8.0,110.0
2,92000,Satış,0,2.0,20.0,51.0
3,435000,Satış,67,0.0,0.0,200.0
5,248000,Satış,130,3.0,17.0,90.0


In [ ]:
df1.drop(['Sale_Type'], axis = 1, inplace = True)
df1.head()

,Price,Location,Number_of_rooms,Floor,KvM_
0,130000,141,2.0,6.0,65.0
1,115000,94,3.0,8.0,110.0
2,92000,0,2.0,20.0,51.0
3,435000,67,0.0,0.0,200.0
5,248000,130,3.0,17.0,90.0


In [ ]:
X=df1.loc[:,['Floor','Number_of_rooms','Location','KvM_']].values
Y=df1['Price'].values

In [ ]:
X[:10]

array([[  6.,   2., 141.,  65.],
       [  8.,   3.,  94., 110.],
       [ 20.,   2.,   0.,  51.],
       [  0.,   0.,  67., 200.],
       [ 17.,   3., 130.,  90.],
       [  0.,   0.,  67., 200.],
       [  4.,   3.,  63.,  55.],
       [  0.,   0.,  67., 220.],
       [ 10.,   2.,  63.,  60.],
       [  0.,   3.,  55.,  70.]])

In [ ]:
len(df1['Location'].unique())

144

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=10)


In [ ]:
xgb=XGBRegressor().fit(X_train,Y_train)


In [ ]:
xgb.__class__.__name__

'XGBRegressor'

In [ ]:
y_pred = xgb.predict(X_test)
dogruluq = r2_score(Y_test, y_pred)
print("Accuracy: {:.4%}".format(dogruluq))



Accuracy: 76.6505%


In [ ]:
location_encoder.classes_

array(['20 Yanvar', '20ci sahə', '8 Noyabr', '8ci kilometr', 'Abşeron',
       'Albalılıq', 'Astara', 'Avtovağzal', 'Azadlıq Prospekti',
       'Ağ şəhər', 'Ağcabədi', 'Ağdaş', 'Badamdar', 'Bahar', 'Bakmil',
       'Bakı', 'Bakıxanov', 'Balakən', 'Balaxanı', 'Bibi Heybət',
       'Bilgəh', 'Biləcəri', 'Biləsuvar', 'Binə', 'Binəqədi', 'Buzovna',
       'Böyükşor', 'Bülbülə', 'Bərdə', 'Ceyranbatan', 'Cəlilabad',
       'Digah', 'Dübəndi', 'Dədə Qorqud', 'Dərnəgül',
       'Elmlər Akademiyası', 'Goranboy', 'Görədil', 'Göygöl', 'Günəşli',
       'Gədəbəy', 'Gənclik', 'Gəncə', 'Hökməli', 'Hövsan', 'Həzi Aslanov',
       'Keşlə', 'Koroğlu', 'Kubinka', 'Köhnə Günəşli', 'Kürdəxanı',
       'Lökbatan', 'Lənkəran', 'M.Ə.Rəsulzadə', 'Masallı', 'Masazır',
       'Massiv A', 'Massiv B', 'Massiv D', 'Massiv G', 'Massiv V',
       'Maştağa', 'Mehdiabad', 'Memar Əcəmi', 'Mingəçevir', 'Müşfiqabad',
       'Məmmədli', 'Mərdəkan', 'NZS', 'Naftalan', 'Nardaran', 'Naxçıvan',
       'Neftçala', 'Neftçilər',

In [ ]:
import joblib

joblib.dump(xgb, '/content/drive/MyDrive/xgbmodel3.joblib')
joblib.dump(location_encoder, '/content/drive/MyDrive/location_encoder2.joblib')

FileNotFoundError: ignored

In [ ]:
X = np.array([[2, 2., 9., 40.]])
predicted_value = xgb.predict(X)
predicted_value


In [ ]:
import joblib
location_encoder = joblib.load("/content/location_encoder.joblib")
location = "Salyan"
location_encoded = location_encoder.transform([location])[0]
print(location_encoded)
type(location_encoded)
float(location_encoded)


In [ ]:
floor = 1.0
rooms = 2.0
location = 3.0
size = 4.0
X = np.array([floor, rooms, location, size])
print(X)
